In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from google.colab import drive
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import timm

In [ ]:
IMAGE_SIZE = 299
BATCH_SIZE = 8
NUM_WORKERS = 2
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

val_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

In [ ]:
drive.mount('/content/drive')
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/dataset-dapa/train/', transform=train_transforms)
val_dataset   = datasets.ImageFolder(root='/content/drive/MyDrive/dataset-dapa/val/',   transform=val_transforms)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)


val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

In [ ]:
if __name__ == "__main__":
    images, labels = next(iter(train_loader))
    print(f"Batch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")

model = timm.create_model('inception_v4', pretrained=True, num_classes=9)

Batch shape: torch.Size([8, 3, 299, 299])
Labels shape: torch.Size([8])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

InceptionV4(
  (features): Sequential(
    (0): ConvNormAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
    (1): ConvNormAct(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
    (2): ConvNormAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
    (3): Mixed3a(
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (c

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        preds = outputs.argmax(dim=1)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        correct += (preds == labels).sum().item()
        total += images.size(0)

    return running_loss / total, correct / total


def validate(model, loader, criterion, device):
    model.eval()
    val_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            preds = outputs.argmax(dim=1)

            val_loss += loss.item() * images.size(0)
            correct += (preds == labels).sum().item()
            total += images.size(0)

    return val_loss / total, correct / total

In [ ]:
num_epochs = 30
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc     = validate(model, val_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}: "
          f"Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"Val   loss {val_loss:.4f}, acc {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "InceptionV4.pth")

Epoch 1/30: Train loss 1.2625, acc 0.5538 | Val   loss 1.2713, acc 0.5960
Epoch 2/30: Train loss 0.8174, acc 0.7175 | Val   loss 0.6044, acc 0.7808
Epoch 3/30: Train loss 0.6723, acc 0.7683 | Val   loss 0.6045, acc 0.8101
Epoch 4/30: Train loss 0.5834, acc 0.8011 | Val   loss 0.3575, acc 0.8939
Epoch 5/30: Train loss 0.5110, acc 0.8264 | Val   loss 0.3912, acc 0.8697
Epoch 6/30: Train loss 0.4766, acc 0.8351 | Val   loss 0.3595, acc 0.8697
Epoch 7/30: Train loss 0.4559, acc 0.8413 | Val   loss 0.3167, acc 0.8949
Epoch 8/30: Train loss 0.4029, acc 0.8588 | Val   loss 0.4199, acc 0.8455
Epoch 9/30: Train loss 0.4048, acc 0.8649 | Val   loss 0.3002, acc 0.9010
Epoch 10/30: Train loss 0.3611, acc 0.8733 | Val   loss 0.3067, acc 0.9020
Epoch 11/30: Train loss 0.3152, acc 0.8930 | Val   loss 0.2095, acc 0.9323
Epoch 12/30: Train loss 0.2966, acc 0.8965 | Val   loss 0.3404, acc 0.8889
Epoch 13/30: Train loss 0.3248, acc 0.8895 | Val   loss 0.2263, acc 0.9293
Epoch 14/30: Train loss 0.2994, ac

In [ ]:
import os

save_dir = '/content/drive/MyDrive/models'
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(save_dir, 'InceptionV4.pth')

torch.save(model.state_dict(), save_path)
print(f"Model saved at: {save_path}")

Model saved at: /content/drive/MyDrive/models/InceptionV4.pth


In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD)
])

In [ ]:
test_dataset = datasets.ImageFolder(
    root='/content/drive/MyDrive/dataset-dapa/test/',
    transform=test_transforms
)

In [ ]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

In [ ]:
def evaluate_on_test(model, test_loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)

            if isinstance(outputs, tuple):
                outputs = outputs[0]

            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    print(classification_report(
        all_labels,
        all_preds,
        target_names=class_names,
        digits=2,
        zero_division=0
    ))


In [ ]:
class_names = test_dataset.classes
evaluate_on_test(model, test_loader, device, class_names)

                     precision    recall  f1-score   support

         algal_spot       0.97      0.97      0.97       170
       brown_blight       0.89      0.87      0.88       134
        gray_blight       0.90      0.91      0.91       163
            healthy       0.99      0.95      0.97       150
         helopeltis       0.94      0.99      0.96       150
           red-rust       0.78      0.88      0.82        24
red-spider-infested       1.00      1.00      1.00        21
           red_spot       0.96      0.94      0.95       172
         white-spot       1.00      0.73      0.84        11

           accuracy                           0.94       995
          macro avg       0.94      0.92      0.92       995
       weighted avg       0.94      0.94      0.94       995



In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

def per_class_accuracy(model, loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)

            if isinstance(outputs, tuple):
                outputs = outputs[0]

            preds = outputs.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    cm = confusion_matrix(all_labels, all_preds)
    print(f"{'Class':<25} {'Accuracy (%)':>12}")
    print("-" * 40)
    for i, class_name in enumerate(class_names):
        correct = cm[i, i]
        total = cm[i].sum()
        acc = 100.0 * correct / total if total > 0 else 0.0
        print(f"{class_name:<25} {acc:>12.2f}")


In [ ]:
class_names = test_dataset.classes
per_class_accuracy(model, test_loader, device, class_names)

Class                     Accuracy (%)
----------------------------------------
algal_spot                       97.06
brown_blight                     87.31
gray_blight                      91.41
healthy                          95.33
helopeltis                       99.33
red-rust                         87.50
red-spider-infested             100.00
red_spot                         94.19
white-spot                       72.73
